In [ ]:
!pip install -q -U kaggle_environments

In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [ ]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def rock_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [ ]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    import random
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


**Напишеи других агентов**

In [ ]:
%%writefile paper_agent.py

# Агент №1: Всегда выбирает "бумагу"
def paper_agent(observation, configuration):
    return 1  # 1 соответствует "бумага"

Overwriting paper_agent.py


In [ ]:
%%writefile scissors_agent.py

# Агент №2: Всегда выбирает "ножницы"
def scissors_agent(observation, configuration):
    return 2  # 2 соответствует "ножницы"

Overwriting scissors_agent.py


In [ ]:
%%writefile random_agent.py

# Агент №3: Выбирает случайное действие
def random_agent(observation, configuration):
    import random
    return random.randrange(0, configuration.signs)  # Случайный выбор от 0 до 2

Overwriting random_agent.py


In [ ]:
%%writefile cycle_agent.py

# Агент №4: Циклически перебирает действия ("камень" -> "бумага" -> "ножницы")
def cycle_agent(observation, configuration):
    return observation.step % configuration.signs  # Цикличность с помощью остатка от деления


Overwriting cycle_agent.py


In [ ]:
%%writefile beat_last_agent.py

# Агент №5: Выбирает действие, побеждающее последний ход оппонента
def beat_last_agent(observation, configuration):
    import random
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % configuration.signs  # Побеждаем последний ход оппонента
    else:
        return random.randrange(0, configuration.signs)  # Случайный выбор на первом ходу



Overwriting beat_last_agent.py


In [ ]:
%%writefile lose_to_last_agent.py

# Агент №6: Выбирает действие, проигрывающее последнему ходу оппонента
def lose_to_last_agent(observation, configuration):
    import random
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % configuration.signs  # Проигрываем последний ход оппонента
    else:
        return random.randrange(0, configuration.signs)  # Случайный выбор на первом ходу


Overwriting lose_to_last_agent.py


In [ ]:
%%writefile even_odd_agent.py

# Агент №7: На чётных ходах "камень", на нечётных "бумага"
def even_odd_agent(observation, configuration):
    if observation.step % 2 == 0:
        return 0  # Чётный ход: "камень"
    else:
        return 1  # Нечётный ход: "бумага"


Overwriting even_odd_agent.py


In [ ]:
%%writefile frequency_agent.py

# Агент №8: Выбирает действие, побеждающее самый частый ход оппонента
def frequency_agent():
    import random
    import numpy as np
    opponent_history = []  # Инициализируем список для хранения истории ходов оппонента

    def agent(observation, configuration):
        if observation.step > 0:
            # Добавляем последний ход оппонента в историю
            opponent_history.append(observation.lastOpponentAction)

        if opponent_history:
            # Считаем, сколько раз оппонент выбирал каждое действие
            counts = np.bincount(opponent_history, minlength=configuration.signs)
            # Находим самое частое действие оппонента
            most_common = np.argmax(counts)
            # Выбираем действие, которое побеждает самое частое действие оппонента
            return int((most_common + 1) % configuration.signs)
        else:
            # На первом ходу выбираем случайное действие
            return int(random.randrange(0, configuration.signs))

    return agent  # Возвращаем функцию-агента

frequency_agent = frequency_agent()  # Присваиваем агенту возвращённую функцию

Overwriting frequency_agent.py


In [ ]:
%%writefile random_two_agent.py

# Агент №9: Случайно выбирает между "камень" и "ножницы"
def random_two_agent(observation, configuration):
    import random
    return random.choice([0, 2])  # Выбор между 0 ("камень") и 2 ("ножницы")


Overwriting random_two_agent.py


In [ ]:
%%writefile adaptive_agent.py

# Агент №10: Меняет действие, если проиграл в предыдущем раунде
def adaptive_agent(observation, configuration):
    import random
    if observation.step > 0 and observation.reward < 0:
        return (observation.lastOpponentAction + 1) % configuration.signs  # Меняем действие
    else:
        return random.randrange(0, configuration.signs)  # Случайный выбор или повтор при победе


Overwriting adaptive_agent.py


In [ ]:
%%writefile mostly_rock_agent.py

# Агент №11: В 90% случаев выбирает "камень", иначе случайное действие
def mostly_rock_agent(observation, configuration):
    import random
    if random.random() < 0.1:
        return random.randrange(0, configuration.signs)  # 10% шанс на случайное действие
    else:
        return 0  # 90% выбирает "камень"


Overwriting mostly_rock_agent.py


In [ ]:
%%writefile counter_beat_last_agent.py

# Агент №12: Предсказывает, что оппонент будет бить наш прошлый ход
def counter_beat_last_agent(observation, configuration):
    import random
    if observation.step > 0:
        predicted_opponent_action = (observation.lastOpponentAction + 1) % configuration.signs  # Предполагаемый ход оппонента
        return (predicted_opponent_action + 1) % configuration.signs  # Выбираем действие, которое победит
    else:
        return random.randrange(0, configuration.signs)  # Случайный выбор на первом ходу


Overwriting counter_beat_last_agent.py


In [ ]:
%%writefile self_lose_agent.py

# Агент №13: Выбирает действие, которое проигрывает нашему прошлому ходу
def self_lose_agent(observation, configuration):
    import random
    if observation.step > 0:
        last_action = observation.lastOpponentAction  # Наш прошлый ход (сохраняется как lastOpponentAction)
        return (last_action + 2) % configuration.signs  # Действие, которое проигрывает нашему прошлому ходу
    else:
        return random.randrange(0, configuration.signs)  # Случайный выбор на первом ходу


Overwriting self_lose_agent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[0, 0]]

In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[-99.0, 99.0]]

**Эксперименты**

In [ ]:
# Список агентов для турнира
agents = [
    ("rock_agent.py", "Rock Agent"),
    ("paper_agent.py", "Paper Agent"),
    ("scissors_agent.py", "Scissors Agent"),
    ("random_agent.py", "Random Agent"),
    ("copy_opponent.py", "Copy Opponent Agent"),
    ("cycle_agent.py", "Cycle Agent"),
    ("beat_last_agent.py", "Beat Last Agent"),
    ("lose_to_last_agent.py", "Lose to Last Agent"),
    ("even_odd_agent.py", "Even Odd Agent"),
    ("frequency_agent.py", "Frequency Agent"),
    ("random_two_agent.py", "Random Two Agent"),
    ("adaptive_agent.py", "Adaptive Agent"),
    ("mostly_rock_agent.py", "Mostly Rock Agent"),
    ("counter_beat_last_agent.py", "Counter Beat Last Agent"),
    ("self_lose_agent.py", "Self Lose Agent")
]

# Инициализируем результаты
scores = {agent[1]: 0 for agent in agents}
results = [] # Инициализируем список для хранения результатов матчей

In [ ]:
# Проводим турнир между агентами
# Проводим турнир
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        agent1_file, agent1_name = agents[i]
        agent2_file, agent2_name = agents[j]

        try:
            # Запускаем матч между двумя агентами
            match_result = evaluate(
                "rps",
                [agent1_file, agent2_file],
                configuration={"episodeSteps": 100},
                num_episodes=1
            )

            # Получаем итоговый счёт для каждого агента
            agent1_score = match_result[0][0]
            agent2_score = match_result[0][1]

            # Определяем победителя и обновляем счёт
            if agent1_score > agent2_score:
                scores[agent1_name] += 1
                winner = agent1_name
            elif agent2_score > agent1_score:
                scores[agent2_name] += 1
                winner = agent2_name
            else:
                winner = "Ничья"

            # Выводим результат матча
            print(f"Матч {agent1_name} vs {agent2_name}: Победитель - {winner}")

            # Сохраняем результат матча
            results.append({
                'Агент 1': agent1_name,
                'Агент 2': agent2_name,
                'Счёт Агент 1': agent1_score,
                'Счёт Агент 2': agent2_score,
                'Победитель': winner
            })

        except Exception as e:
            print(f"Ошибка в матче между {agent1_name} и {agent2_name}: {e}")
            continue

# После турнира выводим общее количество побед для каждого агента
print("\nРезультаты турнира:")
for agent_name, score in scores.items():
    print(f"{agent_name}: {score} побед")

Матч Rock Agent vs Paper Agent: Победитель - Paper Agent
Матч Rock Agent vs Scissors Agent: Победитель - Rock Agent
Матч Rock Agent vs Random Agent: Победитель - Ничья
Матч Rock Agent vs Copy Opponent Agent: Победитель - Ничья
Матч Rock Agent vs Cycle Agent: Победитель - Ничья
Матч Rock Agent vs Beat Last Agent: Победитель - Beat Last Agent
Матч Rock Agent vs Lose to Last Agent: Победитель - Rock Agent
Матч Rock Agent vs Even Odd Agent: Победитель - Even Odd Agent
Матч Rock Agent vs Frequency Agent: Победитель - Frequency Agent
Матч Rock Agent vs Random Two Agent: Победитель - Rock Agent
Матч Rock Agent vs Adaptive Agent: Победитель - Ничья
Матч Rock Agent vs Mostly Rock Agent: Победитель - Ничья
Матч Rock Agent vs Counter Beat Last Agent: Победитель - Rock Agent
Матч Rock Agent vs Self Lose Agent: Победитель - Rock Agent
Матч Paper Agent vs Scissors Agent: Победитель - Scissors Agent
Матч Paper Agent vs Random Agent: Победитель - Ничья
Матч Paper Agent vs Copy Opponent Agent: Победите

In [ ]:
# Определяем лучшую стратегию на основе количества побед
best_agent = max(scores, key=scores.get)
max_wins = scores[best_agent]
print(f"\nЛучшая стратегия по количеству побед: {best_agent} с {max_wins} победами")


Лучшая стратегия по количеству побед: Beat Last Agent с 10 победами


In [ ]:
# Рассчитываем средний счёт для каждого агента
average_scores = {}
for agent_name in scores.keys():
    total_score = 0
    games_played = 0
    for result in results:
        if result['Агент 1'] == agent_name:
            total_score += result['Счёт Агент 1']
            games_played += 1
        elif result['Агент 2'] == agent_name:
            total_score += result['Счёт Агент 2']
            games_played += 1
    if games_played > 0:
        average_scores[agent_name] = total_score / games_played
    else:
        average_scores[agent_name] = 0

# Определяем агента с наивысшим средним счётом
best_average_agent = max(average_scores, key=average_scores.get)
best_average_score = average_scores[best_average_agent]
print(f"\nАгент с наивысшим средним счётом: {best_average_agent} со средним счётом {best_average_score:.2f}")

# Опционально, выводим средние счёты всех агентов
print("\nСредние счёты всех агентов:")
for agent_name, avg_score in average_scores.items():
    print(f"{agent_name}: {avg_score:.2f}")


Агент с наивысшим средним счётом: Beat Last Agent со средним счётом 52.71

Средние счёты всех агентов:
Rock Agent: 6.64
Paper Agent: 17.21
Scissors Agent: -0.64
Random Agent: -1.79
Copy Opponent Agent: -4.93
Cycle Agent: -14.00
Beat Last Agent: 52.71
Lose to Last Agent: -32.93
Even Odd Agent: -2.29
Frequency Agent: 39.71
Random Two Agent: -2.71
Adaptive Agent: -20.00
Mostly Rock Agent: 5.50
Counter Beat Last Agent: -30.14
Self Lose Agent: -12.36
